In [ ]:
!pip install kaggle

In [ ]:
# import necessary packages
import os
import glob
import gc
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import seaborn as sns
from matplotlib.image import imread
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import ResNet50

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification
!kaggle datasets download -d raghavrpotdar/fresh-and-stale-images-of-fruits-and-vegetables

In [ ]:
!unzip fruits-fresh-and-rotten-for-classification.zip
!unzip fresh-and-stale-images-of-fruits-and-vegetables.zip

In [ ]:
fresh_tomato =  '/content/fresh_tomato'
stale_tomato = '/content/stale_tomato'

In [ ]:
train_go, test_go = train_test_split(os.listdir(fresh_tomato), test_size=0.2, random_state=42)
train_gt, test_gt = train_test_split(os.listdir(stale_tomato), test_size=0.2, random_state=42)

In [ ]:
def move_files(file_path, destinnation_folder):
    if not os.path.exists(destinnation_folder):
        os.makedirs(destinnation_folder)
    for file in file_path:
        shutil.copy(file, destinnation_folder)


move_files([fresh_tomato + '/' + i for i in train_go], 'dataset/train/fresh_tomato')
move_files([fresh_tomato + '/' + i for i in test_go], 'dataset/test/fresh_tomato')
move_files([stale_tomato + '/' + i for i in train_gt], 'dataset/train/stale_tomato')
move_files([stale_tomato + '/' + i for i in test_gt], 'dataset/test/stale_tomato')

In [ ]:
from distutils.dir_util import copy_tree

train_folder = '/content/dataset/train';
test_folder =  '/content/dataset/test'

# Make a new train folder with fresh fruits
toDirectory = '/content/working/train/freshfruits';

fromDirectory = train_folder + '/freshapples';
copy_tree(fromDirectory, toDirectory);
fromDirectory = train_folder + '/freshbanana';
copy_tree(fromDirectory, toDirectory);
fromDirectory = train_folder + '/freshoranges';
copy_tree(fromDirectory, toDirectory);
fromDirectory = train_folder + '/fresh_tomato';
copy_tree(fromDirectory, toDirectory);

# Make a new train folder with rotten fruits
toDirectory = '/content/working/train/rottenfruits';

fromDirectory = train_folder + '/rottenapples';
copy_tree(fromDirectory, toDirectory);
fromDirectory = train_folder + '/rottenbanana';
copy_tree(fromDirectory, toDirectory);
fromDirectory = train_folder + '/rottenoranges';
copy_tree(fromDirectory, toDirectory);
fromDirectory = train_folder + '/stale_tomato';
copy_tree(fromDirectory, toDirectory);

# Make a new validation folder with fresh fruits
toDirectory = '/content/working/validation/freshfruits';

fromDirectory = test_folder + '/freshapples';
copy_tree(fromDirectory, toDirectory);
fromDirectory = test_folder + '/freshbanana';
copy_tree(fromDirectory, toDirectory);
fromDirectory = test_folder + '/freshoranges';
copy_tree(fromDirectory, toDirectory);
fromDirectory = test_folder + '/fresh_tomato';
copy_tree(fromDirectory, toDirectory);

# Make a new validation folder with rotten fruits
toDirectory = '/content/working/validation/rottenfruits';

fromDirectory = test_folder + '/rottenapples';
copy_tree(fromDirectory, toDirectory);
fromDirectory = test_folder + '/rottenbanana';
copy_tree(fromDirectory, toDirectory);
fromDirectory = test_folder + '/rottenoranges';
copy_tree(fromDirectory, toDirectory);
fromDirectory = test_folder + '/stale_tomato';
copy_tree(fromDirectory, toDirectory);


In [ ]:
# Dataset generation
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
# Transfert learning
from keras.applications import VGG16
# Optimizer
from keras.optimizers import Adam
# Keras layers
from keras.layers import Input, Dense, Dropout, Flatten, AveragePooling2D
# Keras model
from keras.models import Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(
    samplewise_center=True,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

datagen_valid = ImageDataGenerator(samplewise_center=True)

train_it = datagen_train.flow_from_directory(
    '/content/working/train',
    target_size=(256, 256),  # Changed to 256, 256
    color_mode="rgb",
    class_mode="binary"
)

valid_it = datagen_valid.flow_from_directory(
    '/content/working/validation',
    target_size=(256, 256),  # Changed to 256, 256
    color_mode="rgb",
    class_mode="binary"
)


In [ ]:
# train_it = datagen_train.flow_from_directory(
#     '/content/working/train',
#     target_size=(224, 224),
#     color_mode="rgb",
#     class_mode="binary"
# )

# # load and iterate validation dataset
# valid_it = datagen_valid.flow_from_directory(
#     '/content/working/validation',
#     target_size=(224, 224),
#     color_mode="rgb",
#     class_mode="binary"
# )

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
vgg16.trainable = False

X_input = Input(shape=(256, 256, 3))
X = vgg16(X_input)
X = AveragePooling2D(pool_size=(3, 3), strides=2, padding='valid', name='AvgPool2D')(X)
X = Flatten(name='Flatten')(X)
X = Dense(200, activation='relu', name='Dense1')(X)
X = Dropout(.1)(X)
X = Dense(100, activation='relu', name='Dense2')(X)
X = Dropout(.1)(X)
X = Dense(1, activation='sigmoid', name='Dense3')(X)

model = Model(inputs=X_input, outputs=X, name='Fruit_Classifer')

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Latih model
hist = model.fit(
    train_it,
    validation_data=valid_it,
    epochs=8,
    batch_size=32
)




58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/8
390/390 [==============================] - 304s 745ms/step - loss: 0.2917 - accuracy: 0.9257 - val_loss: 0.0724 - val_accuracy: 0.9735
Epoch 2/8
135/390 [=========>....................] - ETA: 2:46 - loss: 0.1145 - accuracy: 0.9601

In [ ]:
# Setelah selesai melatih model
model.save('fruit_classifier_model.keras')


In [ ]:
# Evaluasi model pada data validasi
validation_loss, validation_accuracy = model.evaluate(valid_it)
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")


In [ ]:
def show_image(image_path):
    image = mpimg.imread(image_path)
    print(image.shape)
    plt.imshow(image)

In [ ]:
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.vgg16 import preprocess_input

def preprocess_image(image_path):
    image = image_utils.load_img(image_path, target_size=(256, 256))  # Adjusted to 256, 256
    image = image_utils.img_to_array(image)
    image = image.reshape(1, 256, 256, 3)  # Adjusted to 256, 256
    image = preprocess_input(image)
    return image

def predict_image(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    if prediction < 0.5:
        print('Fresh')
    else:
        print('Rotten')

predict_image('/content/61fZ+YAYGaL._SL1500_.jpg')


In [ ]:
import pickle
with open("modelFruit_binary.pkl", "wb") as f:
  pickle.dump(model, f)